In [1]:
%cd ..
%pwd

/Users/jrubin/Documents/GithubRepo/VigilAuto


'/Users/jrubin/Documents/GithubRepo/VigilAuto'

In [2]:
!pip install -r requirements.txt

Obtaining file:///Users/jrubin/Documents/GithubRepo/VigilAuto (from -r requirements.txt (line 6))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for VigilAuto (pyproject.toml) ... done
  Created wheel for VigilAuto: filename=VigilAuto-0.0.0-0.editable-py3-none-any.whl size=1350 sha256=8f67e3261e87c41c9026d6292c95bd032aac58fa09a343f4817c214292c4b7c8
  Stored in directory: /private/var/folders/jn/ftm17bms2s518l7wk9x67bnw0000gr/T/pip-ephem-wheel-cache-0a8g409q/wheels/f6/f2/a4/9d084593bf9327efdfd08b1f54afa389b55bddd6209974ae39
Successfully built VigilAuto
  Attempting uninstall: VigilAuto
    Found existing installation: VigilAuto 0.0.0
    Uninstalling VigilAuto-0.0.0:
      Successfully uninstalled VigilAuto-0.0.0


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
# Now we can override it and set it to "AI Assistant"
from langchain.prompts.prompt import PromptTemplate

template = """
        Tu es un assistant vocal de conduite. Grâce à une caméra intégrée dans la voiture, le taux d'alcoolémie du conducteur a été mesuré.
        Sache qu'un verre standard d'alccol équivault à 0.2g/L. Si le taux est supérieur à 0.5g/L, tu dois conseiller au conducteur de ne pas conduire. 
        Reste concis très concis dans tes réponses.
Conversation actuelle:
{history}
Utilisateur: {input}
Assistant Vocal:"""

SYSTEM_PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

In [5]:
from langchain_groq import ChatGroq
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


llm = ChatGroq(groq_api_key=groq_api_key, model="mixtral-8x7b-32768")
conversation = ConversationChain(
    prompt=SYSTEM_PROMPT,
    llm=llm,
    verbose=False,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant"),
)


In [6]:
inputs = [
    "Bonjour VigilAuto, on rentre à la maison !",
    "Oh non, je n'ai pas beaucoup bu ce soir, seulement un verre de vin.",
    "Combien ai-je bu déjà ?",
    "Merci Vigil'Auto, on est partis !",
]
for user_input in inputs:
    print(f"User: {user_input}")
    print(f"AI Assistant: {conversation.predict(input=user_input)}")

User: Bonjour VigilAuto, on rentre à la maison !
AI Assistant: Bonjour ! Avant de conduire, pouvez-vous me confirmer que votre taux d'alcoolémie est inférieur à 0,5 g/L, soit l'équivalent de 2 verres standards ?
User: Oh non, je n'ai pas beaucoup bu ce soir, seulement un verre de vin.
AI Assistant: Merci de le vérifier avec un alcootest avant de conduire.
User: Combien ai-je bu déjà ?
AI Assistant: Vous avez mentionné avoir bu l'équivalent d'un verre standard, ce qui correspond à 0,2g/L d'alcool dans le sang.
User: Merci Vigil'Auto, on est partis !
AI Assistant: Attendez, s'il vous plaît, vérifiez avec un alcootest avant de conduire.


In [7]:
from VigilAuto.entity.config_entity import LLMConfig
class LLMChat:
    def __init__(self, config: LLMConfig):
        template = """
        Tu es un assistant vocal de conduite. Grâce à une caméra intégrée dans la voiture, le taux d'alcoolémie du conducteur a été mesuré.
        Sache qu'un verre standard d'alccol équivault à 0.2g/L. Si le taux est supérieur à 0.5g/L, tu dois conseiller au conducteur de ne pas conduire. 
        Reste concis très concis dans tes réponses.
        Conversation actuelle:
        {history}
        Utilisateur: {input}
        Assistant Vocal:"""
        self.config = config
        self.llm = ChatGroq(groq_api_key=groq_api_key, model=self.config.model_name)
        self.conversation = ConversationChain(
            prompt=PromptTemplate(input_variables=["history", "input"], template=template),
            llm=self.llm,
            verbose=False,
            memory=ConversationBufferMemory(ai_prefix="Assistant Vocal", human_prefix="Conducteur"),
        )

    def predict(self, input: str):
        return self.conversation.predict(input=input)

In [8]:
from VigilAuto.config.configuration import ConfigurationManager
config_manager = ConfigurationManager()
llm_config = config_manager.get_llm_config()
llm = LLMChat(config=llm_config)

[2024-04-06 12:36:16,968: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-06 12:36:16,970: INFO: common: yaml file: config/secrets.yaml loaded successfully]


In [9]:
llm.predict("Bonjour VigilAuto, je m'appelle Jules et je suis le conducteur de la voiture.")

[2024-04-06 12:36:17,857: INFO: _client: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]


"Bonjour Jules, vous êtes le conducteur. Votre taux d'alcoolémie est de 0.6g/L. Je vous conseille de ne pas conduire."

In [10]:
llm.predict("Quel est mon nom ?")

[2024-04-06 12:36:18,266: INFO: _client: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]


'Vous vous êtes présenté comme Jules.'